<center>
<img src='images/pandaearth.jpg' align='center' width="1100x">
</center>

# Андан на экономе

## Семинар 5: Аналитика продаж в pandas

В этот раз мы с вами рассмотрим реальный продуктовый кейс по анализу продаж в некотором онлайн магазине. Поотвечаем на бизнес-вопросы, которые часто волнуют аналитиков в крупных компаниях.

In [ ]:
import pandas as pd 
import numpy as np
import scipy.stats as sts

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
sns.set_theme(style="whitegrid", palette="muted")
plt.rcParams['figure.figsize'] = (8, 4)

### Описание данных

У вас есть данные о продажах в некотором интернет-магазине техники. 

- **`Order ID`** – айдишник заказа;
    - _айдишник является уникальным номером для каждого созданного заказа и определяется в момент создания заказа_
- **`Product`** – товар, который пользователь добавил в свой заказ;
    - _обратите внимание, что в одном заказе может быть несколько товаров_
- **`Quantity Ordered`** – кол-во определенного товара в заказе;
- **`Price Each`** – цена одного определенного товара в заказе (в долл.);
- **`Order Date`** – дата создания заказа;
- **`Purchase Address`** – адрес доставки заказа;
- **`User ID`** – айдишник пользователя, сделавшего заказ.
    - _при этом один заказ относится только к одному пользователю, то есть связка Order ID – User ID является уникальной_

In [ ]:
df = pd.read_csv('data/sales_data.csv')

In [ ]:
df.head()

### Предварительный анализ

Перед тем, как приступать к решению конкретной задачи, нужно посмотреть на ваши данные. Понять, каким образом они устроены, какие в целом признаки у вас есть, правильно ли эти данные подгрузились, есть ли у вас пропуски, какой формат имеет каждая колонка и так далее. 

**Что там по кол-ву наблюдений?**

In [ ]:
df.shape

**Что там с пропусками?**

In [ ]:
df.isnull().sum()

> Видим, что в данных есть пропуски. Причем кажется, что по каждой колонке это одни и те же строки. В таком случае мы можем просто их удалить.

In [ ]:
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)

In [ ]:
# кол-во колонок уменьшилось ровно на 545
df.shape

**Что там по формату колонок?**

In [ ]:
df.info()

> Видим, что все колонки имеют тип данных `object` – это не очень хорошо, ведь мы явно знаем, что `Order ID`, `User ID` и `Quantity Ordered` должны иметь тип `int`, а цена – `float`. При этом в pandas есть специальный формат для даты и по-хорошему нужно им тоже воспользоваться для колонки `Order Date`. Скорей всего такое произошло из-за пропусков, либо из-за других проблем в колонках. Нужно будет найти их и после этого подкорректировать форматы колонок.

In [ ]:
# пробуем перевести колонку Order ID в int, получаем ошибку и разбираемся почему...
df['Order ID'].astype('int64')

In [ ]:
# в колонках почему-то лежат их названия, надо бы их выкинуть
df[~df['Order ID'].str.isdigit()]

In [ ]:
# оставляем только хорошие наблюдения
df = df[df['Order ID'].str.isdigit()]
df.reset_index(drop=True, inplace=True)

In [ ]:
# теперь пробуем перевести тип колонок еще раз
df[['Order ID', 'Quantity Ordered', 'User ID']] = df[['Order ID', 'Quantity Ordered', 'User ID']].astype('float64')
df[['Order ID', 'Quantity Ordered', 'User ID']] = df[['Order ID', 'Quantity Ordered', 'User ID']].astype('int64')
df['Price Each'] = df['Price Each'].astype('float64')

Теперь разберемся с датой:

In [ ]:
df['Order Date'] = pd.to_datetime(df['Order Date'])

In [ ]:
# посмотрим еще раз на тип колонок
df.info()

Отлично!

**Что там с распределением признаков?**

Кол-во заказов

In [ ]:
df['Quantity Ordered'].hist(bins=30);

In [ ]:
df['Quantity Ordered'].hist(bins=30, log=True);

> Похоже на Пуассона

Стоимость заказа

In [ ]:
# готовим данные для графика
df['GMV'] = df['Price Each'] * df['Quantity Ordered']
gb = df.groupby('Order ID')['GMV'].sum()

In [ ]:
gb.hist(bins=30);

In [ ]:
gb.hist(bins=30, log=True);

> Есть явные выбросы в районе $\$2000$ и выше. Не будем пока их удалять, но будем держать в голове.

С помощью метода `.value_counts()` можно еще смотреть на распределение значений в категориальных признаках, например, можем посмотреть, какие товары чаще всего заказывали:

In [ ]:
df['Product'].value_counts()

На первых этапах не стоит сильно упарываться в предварительный анализ, пора переходить к задачам.

### Кейс №1: Реклама и пиар

Предположим, что перед нами стоит задача привлечения новых пользователей и удержание существующих. Для этого мы собираемся, во-первых, устраивать различные промоакции в периоды падения активности, чтобы стимулировать пользователей покупать товары, а во-вторых, запускать крупные рекламные кампании, наоборот, в период высокой активности пользователей, чтобы пользователи покупали товары именно в нашем интернет-магазине. 

__Как определить период, когда следует запускать разного рода акции?__

Обычно, в таких задачах _нету единственного правильного решения_. Вам нужно накидать возможные гипотезы и варианты, а затем обсудить минусы и плюсы каждого подхода. 

При этом нужно всегда отталкиваться от данных, которые у вас есть, и от сроков, которые вы готовы потратить на эту задачу:
- можно построить супер новороченный предсказательный алгоритм и потратить на него полгода, а можно пару часов посидеть в пандасе и уже придумать вполне себе хорошее решение

_Одно из возможных решений:_
1. Посмотреть на динамику кол-ва заказов по дням, возможно увидим сезонность.
2. Посмотреть распределение выручки от заказов по месяцам.
3. Посмотреть, в какое время суток чаще всего создают заказ.

1. Динамика кол-ва заказов по дням

In [ ]:
# youre code

2. Динамика выручки по месяцам

In [ ]:
# youre code

По первым двум графикам можем сказать, что 

> - Активность ниже всего в январе, так как посленовогодние праздники + падает летом
> - Активность выше всего в октябре, апреле и в декабре – самые сезонные месяцы для покупки техники. Черная пятница? Новогодние праздники?

3. Динамика кол-ва заказов от времени суток

In [ ]:
# youre code

По третьему графику видно, что

> Активность заказов начинает расти с самого утра и достигает пика на обед, а также на ранний вечер. Это часы свободного времени пользователей. Скорее всего пользователи вечером возвращаются домой с работы и у них есть время на заказы, либо в обеденное время. 

Что дальше? – Да что угодно! Можно закапываться в данные сколь угодно, пока не достигнете желаемых результатов, главное соблюдать баланс и не анализировать ради анализа.

Например, мы можем посмотреть, в каком городе больше всего заказов и сказать, что баннеры запускать лучше там. Сгенерируйте собственную гипотезу и проверьте ее:

In [ ]:
# youre code again

### Кейс №2: Доверительный интервал для среднего чека

Предположим, что вы хотите понять, а сколько денег вы в среднем получаете с одного сделанного заказа. При этом вы хотите не только получить точечную оценку, посчитав среднее по выборке, но и построить для этого среднего доверительный интервал.

**Попытка №1**

Мы уже знаем, что доверительный интервал для среднего можно построить из ЦПТ:

$$ 
\bar x \overset{asy}{\sim} N \left(\mu, \frac{\hat{\sigma}^2}{n}\right).
$$ 

$$
\bar x_n \pm z_{1 - \frac{\alpha}{2}} \cdot \frac{\hat \sigma}{\sqrt{n}}
$$

Давайте сделаем это

In [ ]:
# считаем, сколько заработали с каждого заказа
# при этом еще отберем user id для каждого заказа (позже будет видно зачем) 
#        – так как связка Order ID – User ID уникальна, можно тупо использовать максимум
byorders = df.groupby('Order ID').agg({'GMV': 'sum', 'User ID': 'max'})

In [ ]:
# строим доверительный интервал
alpha = 0.05
mean_hat, std_hat, n = byorders['GMV'].mean(), byorders['GMV'].std(), byorders.shape[0]


left, right = sts.norm.interval(1 - alpha, loc=mean_hat, scale=std_hat/np.sqrt(n))

delta = right - left
print(f"Среднее по выборке: {mean_hat:.4}")
print(f"Доверительный интервал [{left:.4}, {right:.4}] ширины {delta:.4}")

> Однако, такой доверительный интервал не совсем корректный. Дело в том, что ЦПТ требует _независимость наблюдений_, но эта предпосылка в данном случае перестает быть верной. И вот почему...

**Попытка №2**

Каждый заказ относится к определенному пользователю. При этом есть пользователи, которые склонны тратить больше на площадке, а есть пользователи, которые меньше. Соответственно, если к нам приходит пользователь, готовый тратить много, то средний чек каждого его заказа будет зависимым между собой. Чтобы учесть это, перед тем как строить доверительный интервал для среднего чека, мы можем усреднить этот средний чек для каждого пользователя, а затем уже считать среднее по всей выборке. 

При этом усреднять можно более умно, используя идею _перевзвешивания_. В данном случае мы не будем ее использовать, но про нее можно посмотреть, например, [здесь](https://www.youtube.com/watch?v=z8CqaOQgYcI).

In [ ]:
# усредняем данные для каждого пользователя

# youre code

In [ ]:
# строим доверительный интервал

# youre code

### Кейс №3: Рекомендации товаров

Теперь предположим, что мы хотим повысить средний чек заказа. Для этого, на этапе того, как пользователь добавляет товар в корзину, мы хотим предлагать ему купить еще товары, которые скорее всего его заинтересуют, учитывая тот товар, который он уже добавил в корзину.

**Как нам определить, какие товары рекомендовать конкретному пользователю?**

_Одно из возможных решений_ – посмотреть, какие товары пользователи покупают вместе чаще всего. Их и рекомендовать.

In [ ]:
# youre code

Что дальше? – Да снова что угодно! 

Например, если у нас были бы данные о пользователях, то мы могли бы использовать признаки пользователей в составлении рекомендаций (кол-во его заказов в целом, возраст, ...). Кстати, обычно данные разбросаны в куче таблиц, поэтому вполне возможна такая ситуация, когда информация по товарам лежит в одной таблице, а информация по пользователям в другой таблице. Тогда вам нужно будет воспользоваться еще одним очень полезным приемом – __сджойнить 2 таблицы по ключу__. В pandas это можно сделать с помощью функции `pd.merge` 

### Небольшая история про джойны

Часто хранить информацию в одной таблице бывает довольно неудобно. Когда у вас очень крупная IT-компания, данных настолько много, что если их добавить в одну таблицу, работать с ней будет очень неэффективно по времени и памяти. Поэтому информацию кладут в кучу разных таблиц, а над ними строят специальные *отношения* - так называемые колонки, по которым можно эти таблицы объединять. 

Пример устройства такой структуры хранения данных:

<img src="https://progi.pro/media/main/82/d3/34/82d334f5b4b49d424a2460e796a30cd5.png" height="200" width="600"> 

<img src="https://community.qlik.com/legacyfs/online/87693_all-joins.png" height="400" width="400">

In [ ]:
df1 = pd.DataFrame({'Student': ['Tom', 'Ujin', 'Ann', 'Polina','Sam'],
                    'group': ['01', '02', '02', '01','02']})
df2 = pd.DataFrame({'Name': ['Tom', 'Ujin', 'Ann', 'Polina', 'Kit'],
                    'GPA': ['7.8', '6.4', '8.3', '9', '10']})
display(df1, df2)

In [ ]:
# inner join по умолчанию 
pd.merge(df1, df2, left_on='Student', right_on='Name')

In [ ]:
# left join - оставляем все, что в левой таблице
pd.merge(df1, df2, left_on='Student', right_on='Name', how='left')

In [ ]:
# outer join
pd.merge(df1, df2, left_on='Student', right_on='Name', how='outer')